In [ ]:
import numpy as np
import pandas as pd
import os
import math
import matplotlib.pyplot as plt

%load_ext nb_black

In [ ]:
excel_file = r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\1 LCOA\data.xlsx"

In [ ]:
lifetime = 25  # 使用年限

r = 0.075  # 贴现率
rv = 0.05  # 资产残值率
capacity_factor_ammonia = 8000 / 8760

a = -1
for i in range(lifetime + 1):
    a = a + 1 / ((1 + r) ** i)
a

In [ ]:
def Cal_LCO(
    C_initial,
    C_variable_year,
    C_electricity_year,
    C_fuel_year,
    C_energy_feedstock_year,
    C_water_year,
    C_catalyst_year,
    C_chemical_feedstock_year,
    C_fixed_operation_year,
    Q_year,
    Vr,
    lifetime=lifetime,
    r=r,
):

    # levelized cost of ammonia
    LCO = (C_initial + C_variable_year * a - Vr / (1 + r) ** lifetime) / (
        Q_year * 1000 * a
    )

    CAPEX = (C_initial - Vr / (1 + r) ** lifetime) / (Q_year * 1000 * a)
    LCOA_electricity = C_electricity_year
    LCOA_fuel = C_fuel_year
    LCOA_energy_feedstock = C_energy_feedstock_year
    OPEX = (
        C_variable_year / (Q_year * 1000)
        - LCOA_electricity
        - LCOA_fuel
        - LCOA_energy_feedstock
    )

    return (LCO, CAPEX, LCOA_electricity, LCOA_fuel, LCOA_energy_feedstock, OPEX)

## Province

In [ ]:
Province_data = pd.read_excel(excel_file, sheet_name="Province")

Province = [
    "Anhui",
    "Beijing",
    "Chongqing",
    "Fujian",
    "Gansu",
    "Guangdong",
    "Guangxi",
    "Guizhou",
    "Hainan",
    "Hebei",
    "Heilongjiang",
    "Henan",
    "Hubei",
    "Hunan",
    "InnerMongolia",
    "Jiangsu",
    "Jiangxi",
    "Jilin",
    "Liaoning",
    "Ningxia",
    "Qinghai",
    "Shaanxi",
    "Shandong",
    "Shanghai",
    "Shanxi",
    "Sichuan",
    "Tianjin",
    "Xinjiang",
    "Yunnan",
    "Zhejiang",
    "National",
]
Year = ["2018", "2020", "2025", "2030", "2035", "2040", "2045", "2050", "2055", "2060"]
Year1 = [
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
    "2023",
    "2024",
    "2025",
    "2026",
    "2027",
    "2028",
    "2029",
    "2030",
    "2031",
    "2032",
    "2033",
    "2034",
    "2035",
    "2036",
    "2037",
    "2038",
    "2039",
    "2040",
    "2041",
    "2042",
    "2043",
    "2044",
    "2045",
    "2046",
    "2047",
    "2048",
    "2049",
    "2050",
    "2051",
    "2052",
    "2053",
    "2054",
    "2055",
    "2056",
    "2057",
    "2058",
    "2059",
    "2060",
]

## CG

In [ ]:
CG_data = pd.read_excel(excel_file, sheet_name="CG")
Q_CG_year = 500000 * capacity_factor_ammonia  # CG每年产氨量

# 固定成本
CG_initial = (
    CG_data.iloc[16:26, 2].sum(axis=0)
    / (200000 / (Q_CG_year / capacity_factor_ammonia)) ** 0.7
)
Vr_CG = CG_initial * rv  # 固定资产残值

# 运营成本
CG_fixed_operation = 0.042 * CG_initial  # 假定年维护费用为初始投资的4.2%

# CG的可变运营成本
CG_water = CG_data.iloc[50, 2] * CG_data.iloc[50, 4]
CG_catalyst = np.dot(CG_data.iloc[56, 2], CG_data.iloc[56, 4])
CG_chemical_feedstock = np.dot(CG_data.iloc[51:58, 2], CG_data.iloc[51:58, 4])

# 省级和时间维度，省级维度最后一行为全国层面
CG_electricity_province = [0] * 31
CG_fuel_province = [0] * 31
CG_energy_feedstock_province = [0] * 31
CG_variable_operation_province = [0] * 31

for j in range(31):
    CG_energy_feedstock_province[j] = Province_data.iloc[j + 2, 4] * CG_data.iloc[47, 4]
    CG_fuel_province[j] = Province_data.iloc[j + 2, 3] * CG_data.iloc[48, 4]
    CG_electricity_province[j] = Province_data.iloc[j + 2, 2] * CG_data.iloc[49, 4]
    CG_variable_operation_province[j] = (
        CG_electricity_province[j]
        + CG_fuel_province[j]
        + CG_energy_feedstock_province[j]
        + CG_water
        + CG_catalyst
        + CG_chemical_feedstock
    )

# CG每年每kg合成氨的变动资本总和
CG_variable_province = [0] * 31
for j in range(31):
    CG_variable_province[j] = (
        CG_fixed_operation + CG_variable_operation_province[j] * Q_CG_year * 1000
    )

# CG各期各省每kg的LCOA
LCOA_CG_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOA_CG_results_province_future[j, i] = (
            CG_initial + CG_variable_province[j] * a - Vr_CG / ((1 + r) ** lifetime)
        ) / (Q_CG_year * 1000 * a)

LCOA_CG_results_province_future_df = pd.DataFrame(
    LCOA_CG_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)

# CG 2018年和2060年全国LCOA的成本结构
LCOA_CG_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=CG_initial,
        C_variable_year=CG_variable_province[30],
        C_electricity_year=CG_electricity_province[30],
        C_fuel_year=CG_fuel_province[30],
        C_energy_feedstock_year=CG_energy_feedstock_province[30],
        C_water_year=CG_water,
        C_catalyst_year=CG_catalyst,
        C_chemical_feedstock_year=CG_chemical_feedstock,
        C_fixed_operation_year=CG_fixed_operation,
        Q_year=Q_CG_year,
        Vr=Vr_CG,
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2018"],
)

LCOA_CG_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=CG_initial,
        C_variable_year=CG_variable_province[30],
        C_electricity_year=CG_electricity_province[30],
        C_fuel_year=CG_fuel_province[30],
        C_energy_feedstock_year=CG_energy_feedstock_province[30],
        C_water_year=CG_water,
        C_catalyst_year=CG_catalyst,
        C_chemical_feedstock_year=CG_chemical_feedstock,
        C_fixed_operation_year=CG_fixed_operation,
        Q_year=Q_CG_year,
        Vr=Vr_CG,
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX",],
    columns=["2060"],
)
LCOA_CG_results_province_future_df

## CG+CCS

In [ ]:
CG_CCS_data = pd.read_excel(excel_file, sheet_name="CG_CCS")

# CG+CCS固定资产投资=CG+CCS
Future_data = pd.read_excel(excel_file, sheet_name="Cost")

CG_capture_initial_future = Future_data.iloc[17, 4:14] * 3.70816230293156 * Q_CG_year
CG_storage_initial_future = Future_data.iloc[23, 4:14] * 3.70816230293156 * Q_CG_year
CG_transport_initial_future = Future_data.iloc[20, 4:14] * 3.70816230293156 * Q_CG_year

Carbon_transport_distance_province = [0] * 31
for j in range(31):
    Carbon_transport_distance_province[j] = Province_data.iloc[j + 2, 8]

CG_transport_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        CG_transport_initial_province_future[j, i] = (
            CG_transport_initial_future[i] * Carbon_transport_distance_province[j]
        )

CG_CCS_initial_province_future = np.zeros((31, 10))
Vr_CG_CCS_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        CG_CCS_initial_province_future[j, i] = (
            CG_initial
            + CG_capture_initial_future[i]
            + CG_storage_initial_future[i]
            + CG_transport_initial_province_future[j, i]
        )
        Vr_CG_CCS_province_future[j, i] = CG_CCS_initial_province_future[j, i] * rv

# CG+CCS运营成本
# CG+CCS固定运营成本
CG_capture_fixed_operation_future = [0] * 10
CG_storage_fixed_operation_future = [0] * 10
for i in range(10):
    CG_capture_fixed_operation_future[i] = 0.0367 * CG_capture_initial_future[i]
    CG_storage_fixed_operation_future[i] = 0.04 * CG_storage_initial_future[i]

CG_transport_fixed_operation_province_future = np.zeros((31, 10))
CG_CCS_fixed_operation_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        CG_transport_fixed_operation_province_future[j, i] = (
            0.0222 * CG_transport_initial_province_future[j, i]
        )
        CG_CCS_fixed_operation_province_future[j, i] = (
            CG_capture_fixed_operation_future[i]
            + CG_storage_fixed_operation_future[i]
            + CG_transport_fixed_operation_province_future[j, i]
        )

# CG+CCS可变运营成本
CG_CCS_water = CG_CCS_data.iloc[69, 2] * CG_CCS_data.iloc[69, 4]
CG_CCS_chemical_feedstock = np.dot(
    CG_CCS_data.iloc[70:73, 2], CG_CCS_data.iloc[70:73, 4]
)

CG_CCS_electricity_province = [0] * 31
CG_CCS_variable_operation_province = [0] * 31
for j in range(31):
    CG_CCS_electricity_province[j] = (
        Province_data.iloc[j + 2, 2] * CG_CCS_data.iloc[73, 4]
    )
    CG_CCS_variable_operation_province[j] = (
        CG_CCS_electricity_province[j] + CG_CCS_water + CG_CCS_chemical_feedstock
    )

# CG+CCS每年每kg合成氨的变动资本总和
CG_CCS_variable_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        CG_CCS_variable_province_future[j, i] = (
            CG_CCS_fixed_operation_province_future[j, i]
            + CG_fixed_operation
            + (
                CG_CCS_variable_operation_province[j]
                + CG_variable_operation_province[j]
            )
            * Q_CG_year
            * 1000
        )


# CG+CCS各期各省每kg的LCOA
LCOA_CG_CCS_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOA_CG_CCS_results_province_future[j, i] = (
            CG_CCS_initial_province_future[j, i]
            + CG_CCS_variable_province_future[j, i] * a
            - Vr_CG_CCS_province_future[j, i] / (1 + r) ** lifetime
        ) / (Q_CG_year * 1000 * a)

LCOA_CG_CCS_results_province_future_df = pd.DataFrame(
    LCOA_CG_CCS_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)

# CG+CCS 2018年和2060年全国LCOA的成本结构
LCOA_CG_CCS_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=CG_CCS_initial_province_future[30, 0],
        C_variable_year=CG_CCS_variable_province_future[30, 0],
        C_electricity_year=CG_electricity_province[30]
        + CG_CCS_electricity_province[30],
        C_fuel_year=CG_fuel_province[30],
        C_energy_feedstock_year=CG_energy_feedstock_province[30],
        C_water_year=CG_water + CG_CCS_water,
        C_catalyst_year=CG_catalyst,
        C_chemical_feedstock_year=CG_chemical_feedstock + CG_CCS_chemical_feedstock,
        C_fixed_operation_year=CG_CCS_fixed_operation_province_future[30, 0]
        + CG_fixed_operation,
        Q_year=Q_CG_year,
        Vr=Vr_CG_CCS_province_future[30, 0],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2018"],
)

LCOA_CG_CCS_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=CG_CCS_initial_province_future[30, 9],
        C_variable_year=CG_CCS_variable_province_future[30, 9],
        C_electricity_year=CG_electricity_province[30]
        + CG_CCS_electricity_province[30],
        C_fuel_year=CG_fuel_province[30],
        C_energy_feedstock_year=CG_energy_feedstock_province[30],
        C_water_year=CG_water + CG_CCS_water,
        C_catalyst_year=CG_catalyst,
        C_chemical_feedstock_year=CG_chemical_feedstock + CG_CCS_chemical_feedstock,
        C_fixed_operation_year=CG_CCS_fixed_operation_province_future[30, 9]
        + CG_fixed_operation,
        Q_year=Q_CG_year,
        Vr=Vr_CG_CCS_province_future[30, 9],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2060"],
)
LCOA_CG_CCS_results_province_future_df

## SMR

In [ ]:
SMR_data = pd.read_excel(excel_file, sheet_name="SMR")
Q_SMR_year = 500000 * capacity_factor_ammonia  # 天然气制氨年产量为50万吨

# 固定成本
SMR_initial = SMR_data.iloc[16:27, 2].sum(axis=0)  # 固定资本
Vr_SMR = SMR_initial * rv  # 固定资产残值

# 运营成本
# SMR的年固定变动资本
SMR_fixed_operation = 0.0282 * SMR_initial

# SMR的可变运营成本
SMR_water = SMR_data.iloc[52, 2] * SMR_data.iloc[52, 4]
SMR_catalyst = np.dot(SMR_data.iloc[53:60, 2], SMR_data.iloc[53:60, 4])
SMR_chemical_feedstock = 0

SMR_electricity_province = [0] * 31
SMR_fuel_province = [0] * 31
SMR_energy_feedstock_province = [0] * 31
SMR_variable_operation_province = [0] * 31

for j in range(31):
    SMR_fuel_province[j] = Province_data.iloc[j + 2, 5] * SMR_data.iloc[50, 4]
    SMR_energy_feedstock_province[j] = (
        Province_data.iloc[j + 2, 5] * SMR_data.iloc[49, 4]
    )

    SMR_electricity_province[j] = Province_data.iloc[j + 2, 2] * SMR_data.iloc[51, 4]
    SMR_variable_operation_province[j] = (
        SMR_fuel_province[j]
        + SMR_energy_feedstock_province[j]
        + SMR_electricity_province[j]
        + SMR_water
        + SMR_catalyst
        + SMR_chemical_feedstock
    )

# SMR每年每kg合成氨的变动资本总和
SMR_variable_province = [0] * 31
for j in range(31):
    SMR_variable_province[j] = (SMR_fixed_operation) + (
        (SMR_variable_operation_province[j]) * Q_SMR_year * 1000
    )

# SMR各期各省每kg的LCOA
LCOA_SMR_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOA_SMR_results_province_future[j, i] = (
            SMR_initial + SMR_variable_province[j] * a - Vr_SMR / ((1 + r) ** lifetime)
        ) / (Q_SMR_year * 1000 * a)

LCOA_SMR_results_province_future_df = pd.DataFrame(
    LCOA_SMR_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)

# SMR 2018年和2060年全国LCOA的成本结构
LCOA_SMR_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=SMR_initial,
        C_variable_year=SMR_variable_province[30],
        C_electricity_year=SMR_electricity_province[30],
        C_fuel_year=SMR_fuel_province[30],
        C_energy_feedstock_year=SMR_energy_feedstock_province[30],
        C_water_year=SMR_water,
        C_catalyst_year=SMR_catalyst,
        C_chemical_feedstock_year=SMR_chemical_feedstock,
        C_fixed_operation_year=SMR_fixed_operation,
        Q_year=Q_SMR_year,
        Vr=Vr_SMR,
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX",],
    columns=["2018"],
)

LCOA_SMR_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=SMR_initial,
        C_variable_year=SMR_variable_province[30],
        C_electricity_year=SMR_electricity_province[30],
        C_fuel_year=SMR_fuel_province[30],
        C_energy_feedstock_year=SMR_energy_feedstock_province[30],
        C_water_year=SMR_water,
        C_catalyst_year=SMR_catalyst,
        C_chemical_feedstock_year=SMR_chemical_feedstock,
        C_fixed_operation_year=SMR_fixed_operation,
        Q_year=Q_SMR_year,
        Vr=Vr_SMR,
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2060"],
)

LCOA_SMR_results_province_future_df

## SMR+CCS

In [ ]:
SMR_CCS_data = pd.read_excel(excel_file, sheet_name="SMR_CCS")

# SMR+CCS固定资产投资=SMR+CCS
Future_data = pd.read_excel(excel_file, sheet_name="Cost")

SMR_capture_initial_future = Future_data.iloc[17, 4:14] * 1.3585 * Q_SMR_year
SMR_storage_initial_future = Future_data.iloc[23, 4:14] * 1.3585 * Q_SMR_year
SMR_transport_initial_future = Future_data.iloc[20, 4:14] * 1.3585 * Q_SMR_year

Carbon_transport_distance_province = [0] * 31
for j in range(31):
    Carbon_transport_distance_province[j] = Province_data.iloc[j + 2, 8]

SMR_transport_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        SMR_transport_initial_province_future[j, i] = (
            SMR_transport_initial_future[i] * Carbon_transport_distance_province[j]
        )

SMR_CCS_initial_province_future = np.zeros((31, 10))
Vr_SMR_CCS_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        SMR_CCS_initial_province_future[j, i] = (
            SMR_initial
            + SMR_capture_initial_future[i]
            + SMR_storage_initial_future[i]
            + SMR_transport_initial_province_future[j, i]
        )
        Vr_SMR_CCS_province_future[j, i] = SMR_CCS_initial_province_future[j, i] * rv

# SMR+CCS运营成本
# SMR+CCS固定运营成本
SMR_capture_fixed_operation_future = [0] * 10
SMR_storage_fixed_operation_future = [0] * 10
for j in range(10):
    SMR_capture_fixed_operation_future[i] = 0.0367 * SMR_capture_initial_future[i]
    SMR_storage_fixed_operation_future[i] = 0.04 * SMR_storage_initial_future[i]

SMR_transport_fixed_operation_province_future = np.zeros((31, 10))
SMR_CCS_fixed_operation_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        SMR_transport_fixed_operation_province_future[j, i] = (
            0.0222 * SMR_transport_initial_province_future[j, i]
        )
        SMR_CCS_fixed_operation_province_future[j, i] = (
            SMR_capture_fixed_operation_future[i]
            + SMR_storage_fixed_operation_future[i]
            + SMR_transport_fixed_operation_province_future[j, i]
        )

# SMR+CCS可变运营成本
SMR_CCS_water = SMR_CCS_data.iloc[76, 2] * SMR_CCS_data.iloc[76, 4]
SMR_CCS_chemical_feedstock = np.dot(
    SMR_CCS_data.iloc[77:80, 2], SMR_CCS_data.iloc[77:80, 4]
)

SMR_CCS_electricity_province = [0] * 31
SMR_CCS_variable_operation_province = [0] * 31
for j in range(31):
    SMR_CCS_electricity_province[j] = (
        Province_data.iloc[j + 2, 2] * SMR_CCS_data.iloc[80, 4]
    )
    SMR_CCS_variable_operation_province[j] = (
        SMR_CCS_electricity_province[j] + SMR_CCS_water + SMR_CCS_chemical_feedstock
    )

# SMR+CCS每年每kg合成氨的变动资本总和
SMR_CCS_variable_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        SMR_CCS_variable_province_future[j, i] = (
            SMR_CCS_fixed_operation_province_future[j, i] + SMR_fixed_operation
        ) + (
            (
                SMR_CCS_variable_operation_province[j]
                + SMR_variable_operation_province[j]
            )
            * Q_SMR_year
            * 1000
        )

# SMR+CCS各期各省每kg的LCOA
LCOA_SMR_CCS_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOA_SMR_CCS_results_province_future[j, i] = (
            SMR_CCS_initial_province_future[j, i]
            + SMR_CCS_variable_province_future[j, i] * a
            - Vr_SMR_CCS_province_future[j, i] / (1 + r) ** lifetime
        ) / (Q_SMR_year * 1000 * a)

LCOA_SMR_CCS_results_province_future_df = pd.DataFrame(
    LCOA_SMR_CCS_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)

# SMR+CCS 2018年和2060年全国LCOA的成本结构
LCOA_SMR_CCS_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=SMR_CCS_initial_province_future[30, 0],
        C_variable_year=SMR_CCS_variable_province_future[30, 0],
        C_electricity_year=SMR_electricity_province[30]
        + SMR_CCS_electricity_province[30],
        C_fuel_year=SMR_fuel_province[30],
        C_energy_feedstock_year=SMR_energy_feedstock_province[30],
        C_water_year=SMR_water + SMR_CCS_water,
        C_catalyst_year=SMR_catalyst,
        C_chemical_feedstock_year=SMR_chemical_feedstock + SMR_CCS_chemical_feedstock,
        C_fixed_operation_year=SMR_CCS_fixed_operation_province_future[30, 0]
        + SMR_fixed_operation,
        Q_year=Q_SMR_year,
        Vr=Vr_SMR_CCS_province_future[30, 0],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX",],
    columns=["2018"],
)

LCOA_SMR_CCS_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=SMR_CCS_initial_province_future[30, 9],
        C_variable_year=SMR_CCS_variable_province_future[30, 9],
        C_electricity_year=SMR_electricity_province[30]
        + SMR_CCS_electricity_province[30],
        C_fuel_year=SMR_fuel_province[30],
        C_energy_feedstock_year=SMR_energy_feedstock_province[30],
        C_water_year=SMR_water + SMR_CCS_water,
        C_catalyst_year=SMR_catalyst,
        C_chemical_feedstock_year=SMR_chemical_feedstock + SMR_CCS_chemical_feedstock,
        C_fixed_operation_year=SMR_CCS_fixed_operation_province_future[30, 9]
        + SMR_fixed_operation,
        Q_year=Q_SMR_year,
        Vr=Vr_SMR_CCS_province_future[30, 9],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX",],
    columns=["2060"],
)
LCOA_SMR_CCS_results_province_future_df

## PV

In [ ]:
# PV
PV_data = pd.read_excel(excel_file, sheet_name="PV")

PV_initial_future = Future_data.iloc[2, 4:14] * 1000

Vr_PV_future = PV_initial_future * rv  # 固定资产残值
PV_variable_year_future = PV_initial_future * 0.03

PV_capacity_factor_province = [0] * 31
Q_PV_year_province = [0] * 31

for j in range(31):
    PV_capacity_factor_province[j] = Province_data.iloc[j + 2, 6]
    Q_PV_year_province[j] = 365 * 24 * PV_capacity_factor_province[j]

LCOE_PV_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOE_PV_results_province_future[j, i] = (
            PV_initial_future[i]
            + PV_variable_year_future[i] * a
            - Vr_PV_future[i] / (1 + r) ** lifetime
        ) / (Q_PV_year_province[j] * 1000 * a)


LCOE_PV_results_province_future_df = pd.DataFrame(
    LCOE_PV_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)
LCOE_PV_results_province_future_df

In [ ]:
LCOE_PV_CAPEX_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOE_PV_CAPEX_results_province_future[j, i] = (
            PV_initial_future[i]
            #             + PV_variable_year_future[i] * a
            - Vr_PV_future[i] / (1 + r) ** lifetime
        ) / (Q_PV_year_province[j] * 1000 * a)


LCOE_PV_CAPEX_results_province_future_df = pd.DataFrame(
    LCOE_PV_CAPEX_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)
LCOE_PV_CAPEX_results_province_future_df

In [ ]:
PV_Q_Green_year_province = [0] * 31
for j in range(31):
    PV_Q_Green_year_province[j] = 100000 * PV_capacity_factor_province[j]  # 绿氨年产能10万吨

PV_capacity_province = [0] * 31
for j in range(31):
    PV_capacity_province[j] = (
        PV_Q_Green_year_province[j]
        / (PV_capacity_factor_province[j])
        * (51.8 * 0.18878 + 0.11 * 0.87445 + 0.47475)
        / 8760
    )

PV_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        PV_initial_province_future[j, i] = (
            Future_data.iloc[2, 4 + i] * 1000 * PV_capacity_province[j]
        )

PV_capacity_electrolyser_AWE_province = [0] * 31
for j in range(31):
    PV_capacity_electrolyser_AWE_province[j] = (
        (
            (PV_Q_Green_year_province[j] / (PV_capacity_factor_province[j]))
            * 0.18878
            * 1000
            / (1000 / 51.8)
        )
        / 8760
        * 1000
    )

PV_AWE_initial_electrolyser_stack_province_future = np.zeros((31, 10))
PV_AWE_initial_electrolyser_BoP_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        PV_AWE_initial_electrolyser_stack_province_future[j, i] = (
            Future_data.iloc[8, i + 4]
            / 2
            * 5000
            / (5000 / PV_capacity_electrolyser_AWE_province[j]) ** 0.88
        ) * 2.5
        PV_AWE_initial_electrolyser_BoP_province_future[j, i] = (
            Future_data.iloc[8, i + 4]
            / 2
            * 5000
            / (5000 / PV_capacity_electrolyser_AWE_province[j]) ** 0.7
        )

PV_capacity_NH3_province = [0] * 31
PV_capacity_N2_province = [0] * 31
for j in range(31):
    PV_capacity_NH3_province[j] = (
        PV_Q_Green_year_province[j] / PV_capacity_factor_province[j]
    )
    PV_capacity_N2_province[j] = (
        0.87445 * PV_capacity_NH3_province[j]
    )  # 每千克合成氨需要投入0.87445kg氮气

initial_ASU_province = [0] * 31
initial_HB_province = [0] * 31

for j in range(31):
    initial_ASU_province[j] = PV_data.iloc[5, 2]
    initial_HB_province[j] = PV_data.iloc[6, 2]

PV_AWE_initial_province_future = np.zeros((31, 10))
PV_Vr_AWE_province_future = np.zeros((31, 10))
PV_AWE_non_PV_fixed_operation_province_future = np.zeros((31, 10))
PV_AWE_PV_fixed_operation_province_future = np.zeros((31, 10))
PV_AWE_fixed_operation_province_future = np.zeros((31, 10))

PV_AWE_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        PV_AWE_initial_province_future[j, i] = (
            PV_AWE_initial_electrolyser_stack_province_future[j, i]
            + PV_AWE_initial_electrolyser_BoP_province_future[j, i]
            + initial_HB_province[j]
            + initial_ASU_province[j]
            + PV_initial_province_future[j, i]
        )
        PV_Vr_AWE_province_future[j, i] = (
            PV_AWE_initial_province_future[j, i] * rv
        )  # 固定资产残值
        PV_AWE_non_PV_fixed_operation_province_future[j, i] = 0.025 * (
            PV_AWE_initial_province_future[j, i] - PV_initial_province_future[j, i]
        )
        PV_AWE_PV_fixed_operation_province_future[j, i] = (
            0.03 * PV_initial_province_future[j, i]
        )
        PV_AWE_fixed_operation_province_future[j, i] = (
            PV_AWE_non_PV_fixed_operation_province_future[j, i]
            + PV_AWE_PV_fixed_operation_province_future[j, i]
        )

AWE_water = PV_data.iloc[121, 2] * PV_data.iloc[139, 2]
AWE_catalyst = PV_data.iloc[122, 2] * PV_data.iloc[140, 2]
AWE_chemical_feedstock = PV_data.iloc[123, 2] * PV_data.iloc[141, 2]

PV_AWE_electricity_province_future = np.zeros((31, 10))
PV_AWE_variable_operation_province_future = np.zeros((31, 10))
PV_AWE_variable_province_future = np.zeros((31, 10))

for i in range(10):
    for j in range(31):
        PV_AWE_electricity_province_future[j, i] = (
            PV_data.iloc[138, 2] * LCOE_PV_results_province_future[j, i]
        )

        PV_AWE_variable_operation_province_future[j, i] = (
            #            PV_AWE_electricity_province_future[j, i]
            +AWE_water
            + AWE_catalyst
            + AWE_chemical_feedstock
        )

        PV_AWE_variable_province_future[j, i] = (
            PV_AWE_fixed_operation_province_future[j, i]
        ) + (
            (PV_AWE_variable_operation_province_future[j, i])
            * PV_Q_Green_year_province[j]
            * 1000
        )

LCOA_PV_AWE_results_province_future = np.zeros((31, 10))

for i in range(10):
    for j in range(31):
        LCOA_PV_AWE_results_province_future[j, i] = (
            PV_AWE_initial_province_future[j, i]
            + PV_AWE_variable_province_future[j, i] * a
            - PV_Vr_AWE_province_future[j, i] / (1 + r) ** lifetime
        ) / (PV_Q_Green_year_province[j] * 1000 * a)

LCOA_PV_AWE_results_province_future_df = pd.DataFrame(
    LCOA_PV_AWE_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)


LCOA_PV_AWE_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=PV_AWE_initial_province_future[30, 0],
        C_variable_year=PV_AWE_variable_province_future[30, 0],
        C_electricity_year=0,  # PV_AWE_electricity_province_future[30, 0],
        C_fuel_year=0,
        C_energy_feedstock_year=0,
        C_water_year=AWE_water,
        C_catalyst_year=AWE_catalyst,
        C_chemical_feedstock_year=0,
        C_fixed_operation_year=PV_AWE_fixed_operation_province_future[30, 0],
        Q_year=PV_Q_Green_year_province[30],
        Vr=PV_Vr_AWE_province_future[30, 0],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2018"],
)

LCOA_PV_AWE_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=PV_AWE_initial_province_future[30, 9],
        C_variable_year=PV_AWE_variable_province_future[30, 9],
        C_electricity_year=0,  # PV_AWE_electricity_province_future[30, 9],
        C_fuel_year=0,
        C_energy_feedstock_year=0,
        C_water_year=AWE_water,
        C_catalyst_year=AWE_catalyst,
        C_chemical_feedstock_year=0,
        C_fixed_operation_year=PV_AWE_fixed_operation_province_future[30, 9],
        Q_year=PV_Q_Green_year_province[30],
        Vr=PV_Vr_AWE_province_future[30, 9],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2060"],
)

LCOA_PV_AWE_results_province_future_df

## Wind

In [ ]:
# Wind
Wind_data = pd.read_excel(excel_file, sheet_name="Wind")

Wind_initial_future = Future_data.iloc[5, 4:14] * 1000

Vr_Wind_future = Wind_initial_future * rv  # 固定资产残值
Wind_variable_year_future = Wind_initial_future * 0.02

Wind_capacity_factor_province = [0] * 31
Q_Wind_year_province = [0] * 31

for j in range(31):
    Wind_capacity_factor_province[j] = Province_data.iloc[j + 2, 7]
    Q_Wind_year_province[j] = 365 * 24 * Wind_capacity_factor_province[j]

LCOE_Wind_results_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        LCOE_Wind_results_province_future[j, i] = (
            Wind_initial_future[i]
            + Wind_variable_year_future[i] * a
            - Vr_Wind_future[i] / (1 + r) ** lifetime
        ) / (Q_Wind_year_province[j] * 1000 * a)
LCOE_Wind_results_province_future_df = pd.DataFrame(
    LCOE_Wind_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)
LCOE_Wind_results_province_future_df

In [ ]:
Wind_Q_Green_year_province = [0] * 31
for j in range(31):
    Wind_Q_Green_year_province[j] = 100000 * Wind_capacity_factor_province[j]  # 绿氨年产能10万吨

Wind_capacity_province = [0] * 31
for j in range(31):
    Wind_capacity_province[j] = (
        Wind_Q_Green_year_province[j]
        / (Wind_capacity_factor_province[j])
        * (51.8 * 0.18878 + 0.11 * 0.87445 + 0.47475)
        / 8760
    )

Wind_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        Wind_initial_province_future[j, i] = (
            Future_data.iloc[5, 4 + i] * 1000 * Wind_capacity_province[j]
        )

Wind_capacity_electrolyser_AWE_province = [0] * 31
for j in range(31):
    Wind_capacity_electrolyser_AWE_province[j] = (
        (
            (Wind_Q_Green_year_province[j] / (Wind_capacity_factor_province[j]))
            * 0.18878
            * 1000
            / (1000 / 51.8)
        )
        / 8760
        * 1000
    )

Wind_AWE_initial_electrolyser_stack_province_future = np.zeros((31, 10))
Wind_AWE_initial_electrolyser_BoP_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        Wind_AWE_initial_electrolyser_stack_province_future[j, i] = (
            Future_data.iloc[8, i + 4]
            / 2
            * 5000
            / (5000 / Wind_capacity_electrolyser_AWE_province[j]) ** 0.88
        ) * 2.5
        Wind_AWE_initial_electrolyser_BoP_province_future[j, i] = (
            Future_data.iloc[8, i + 4]
            / 2
            * 5000
            / (5000 / Wind_capacity_electrolyser_AWE_province[j]) ** 0.7
        )

Wind_capacity_NH3_province = [0] * 31
Wind_capacity_N2_province = [0] * 31
for j in range(31):
    Wind_capacity_NH3_province[j] = (
        Wind_Q_Green_year_province[j] / Wind_capacity_factor_province[j]
    )
    Wind_capacity_N2_province[j] = (
        0.87445 * Wind_capacity_NH3_province[j]
    )  # 每千克合成氨需要投入0.87445kg氮气

initial_ASU_province = [0] * 31
initial_HB_province = [0] * 31

for j in range(31):
    initial_ASU_province[j] = Wind_data.iloc[5, 2]
    initial_HB_province[j] = Wind_data.iloc[6, 2]

Wind_AWE_initial_province_future = np.zeros((31, 10))
Wind_Vr_AWE_province_future = np.zeros((31, 10))
Wind_AWE_non_Wind_fixed_operation_province_future = np.zeros((31, 10))
Wind_AWE_Wind_fixed_operation_province_future = np.zeros((31, 10))
Wind_AWE_fixed_operation_province_future = np.zeros((31, 10))

Wind_AWE_initial_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        Wind_AWE_initial_province_future[j, i] = (
            Wind_AWE_initial_electrolyser_stack_province_future[j, i]
            + Wind_AWE_initial_electrolyser_BoP_province_future[j, i]
            + initial_HB_province[j]
            + initial_ASU_province[j]
            + Wind_initial_province_future[j, i]
        )
        Wind_Vr_AWE_province_future[j, i] = (
            Wind_AWE_initial_province_future[j, i] * rv
        )  # 固定资产残值
        Wind_AWE_non_Wind_fixed_operation_province_future[j, i] = 0.025 * (
            Wind_AWE_initial_province_future[j, i] - Wind_initial_province_future[j, i]
        )
        Wind_AWE_Wind_fixed_operation_province_future[j, i] = (
            0.02 * Wind_initial_province_future[j, i]
        )
        Wind_AWE_fixed_operation_province_future[j, i] = (
            Wind_AWE_non_Wind_fixed_operation_province_future[j, i]
            + Wind_AWE_Wind_fixed_operation_province_future[j, i]
        )

AWE_water = Wind_data.iloc[121, 2] * Wind_data.iloc[139, 2]
AWE_catalyst = Wind_data.iloc[122, 2] * Wind_data.iloc[140, 2]
AWE_chemical_feedstock = Wind_data.iloc[123, 2] * Wind_data.iloc[141, 2]

Wind_AWE_electricity_province_future = np.zeros((31, 10))
Wind_AWE_variable_operation_province_future = np.zeros((31, 10))
Wind_AWE_variable_province_future = np.zeros((31, 10))

for i in range(10):
    for j in range(31):
        Wind_AWE_electricity_province_future[j, i] = (
            Wind_data.iloc[138, 2] * LCOE_Wind_results_province_future[j, i]
        )

        Wind_AWE_variable_operation_province_future[j, i] = (
            #            Wind_AWE_electricity_province_future[j, i]
            +AWE_water
            + AWE_catalyst
            + AWE_chemical_feedstock
        )

        Wind_AWE_variable_province_future[j, i] = (
            Wind_AWE_fixed_operation_province_future[j, i]
        ) + (
            (Wind_AWE_variable_operation_province_future[j, i])
            * Wind_Q_Green_year_province[j]
            * 1000
        )

LCOA_Wind_AWE_results_province_future = np.zeros((31, 10))

for i in range(10):
    for j in range(31):
        LCOA_Wind_AWE_results_province_future[j, i] = (
            Wind_AWE_initial_province_future[j, i]
            + Wind_AWE_variable_province_future[j, i] * a
            - Wind_Vr_AWE_province_future[j, i] / (1 + r) ** lifetime
        ) / (Wind_Q_Green_year_province[j] * 1000 * a)

LCOA_Wind_AWE_results_province_future_df = pd.DataFrame(
    LCOA_Wind_AWE_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)


LCOA_Wind_AWE_results_2018 = pd.DataFrame(
    Cal_LCO(
        C_initial=Wind_AWE_initial_province_future[30, 0],
        C_variable_year=Wind_AWE_variable_province_future[30, 0],
        C_electricity_year=0,  # Wind_AWE_electricity_province_future[30, 0],
        C_fuel_year=0,
        C_energy_feedstock_year=0,
        C_water_year=AWE_water,
        C_catalyst_year=AWE_catalyst,
        C_chemical_feedstock_year=0,
        C_fixed_operation_year=Wind_AWE_fixed_operation_province_future[30, 0],
        Q_year=Wind_Q_Green_year_province[30],
        Vr=Wind_Vr_AWE_province_future[30, 0],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2018"],
)

LCOA_Wind_AWE_results_2060 = pd.DataFrame(
    Cal_LCO(
        C_initial=Wind_AWE_initial_province_future[30, 9],
        C_variable_year=Wind_AWE_variable_province_future[30, 9],
        C_electricity_year=0,  # Wind_AWE_electricity_province_future[30, 9],
        C_fuel_year=0,
        C_energy_feedstock_year=0,
        C_water_year=AWE_water,
        C_catalyst_year=AWE_catalyst,
        C_chemical_feedstock_year=0,
        C_fixed_operation_year=Wind_AWE_fixed_operation_province_future[30, 9],
        Q_year=Wind_Q_Green_year_province[30],
        Vr=Wind_Vr_AWE_province_future[30, 9],
    ),
    index=["LCO", "CAPEX", "Electricity", "Fuel", "Energy_feedstock", "OPEX"],
    columns=["2060"],
)

LCOA_Wind_AWE_results_province_future_df

# integrated

In [ ]:
# electricity
PV_capacity_province_future = pd.read_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\1 LCOA\capacity_PV.xlsx"
).iloc[:, 1:]
Wind_capacity_province_future = pd.read_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\1 LCOA\capacity_Wind.xlsx"
).iloc[:, 1:]

PV_cost_province_future = np.zeros((30, 10))
Wind_cost_province_future = np.zeros((30, 10))
Vr_PV_cost_province_future = np.zeros((30, 10))
Vr_Wind_cost_province_future = np.zeros((30, 10))
PV_variable_year_future = np.zeros((30, 10))
Wind_variable_year_future = np.zeros((30, 10))

for i in range(10):
    for j in range(30):
        PV_cost_province_future[j, i] = (
            PV_initial_future[i] * PV_capacity_province_future.iloc[j, i] / 1000
        )
        Wind_cost_province_future[j, i] = (
            Wind_initial_future[i] * Wind_capacity_province_future.iloc[j, i] / 1000
        )
        Vr_PV_cost_province_future[j, i] = PV_cost_province_future[j, i] * rv
        Vr_Wind_cost_province_future[j, i] = PV_cost_province_future[j, i] * rv
        PV_variable_year_future[j, i] = PV_cost_province_future[j, i] * 0.03
        Wind_variable_year_future[j, i] = Wind_cost_province_future[j, i] * 0.02

Q_electricity = 100000000 * (0.18878 * 51.8 + 0.877445 * 1.1 + 0.47473)

LCOE_results_province_future = np.zeros((31, 10))

for i in range(10):
    for j in range(30):
        LCOE_results_province_future[j, i] = (
            PV_cost_province_future[j, i]
            + Wind_cost_province_future[j, i]
            + (PV_variable_year_future[j, i] + Wind_variable_year_future[j, i]) * a
            - (Vr_PV_cost_province_future[j, i] + Vr_Wind_cost_province_future[j, i])
            / (1 + r) ** lifetime
        ) / (Q_electricity * a)
LCOE_results_province_future[30, :] = np.mean(
    LCOE_results_province_future[:30, :], axis=0
)

electricity_province_future = np.zeros((31, 10))
for i in range(10):
    for j in range(31):
        electricity_province_future[j, i] = LCOE_results_province_future[j, i] * (
            0.18878 * 51.8 + 0.877445 * 0.11 + 0.47473
        )

LCOE_results_province_future_df = pd.DataFrame(
    LCOE_results_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)
electricity_province_future_df = pd.DataFrame(
    electricity_province_future,
    index=Province,
    columns=[
        "2018",
        "2020",
        "2025",
        "2030",
        "2035",
        "2040",
        "2045",
        "2050",
        "2055",
        "2060",
    ],
)

In [ ]:
LCOA_integrated_results_province_future_18_60_df = pd.read_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\1 LCOA\integrated_cost.xlsx"
).iloc[:, 1:]

In [ ]:
# 每五年线性平均至每年
def calculate_lcoa_results(input_df):
    lcoa_results = np.zeros((31, 43))
    lcoa_results[:, 0] = input_df.iloc[:, 0]
    lcoa_results[:, 1] = (input_df.iloc[:, 0] + input_df.iloc[:, 1]) / 2

    for a in range(8):
        for i in range(5):
            yearly_average = (input_df.iloc[:, a + 2] - input_df.iloc[:, a + 1]) / 5
            lcoa_results[:, i + 5 * a + 2] = (
                yearly_average * i + input_df.iloc[:, a + 1]
            )

    lcoa_results[:, 42] = input_df.iloc[:, 9]

    return pd.DataFrame(lcoa_results)


# Calculate LCOA results for different dataframes
LCOA_CG_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_CG_results_province_future_df
)
LCOA_CG_CCS_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_CG_CCS_results_province_future_df
)
LCOA_SMR_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_SMR_results_province_future_df
)
LCOA_SMR_CCS_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_SMR_CCS_results_province_future_df
)
LCOA_PV_AWE_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_PV_AWE_results_province_future_df
)
LCOA_Wind_AWE_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_Wind_AWE_results_province_future_df
)
LCOA_integrated_results_province_future_18_60 = pd.read_excel(
    r"D:\2 Master\2 Paper\2 Ammonia\0 press\SI\1 LCOA\integrated_cost.xlsx"
).iloc[0:31, 1:]
LCOA_integrated_results_province_future_18_60_df = calculate_lcoa_results(
    LCOA_integrated_results_province_future_18_60
)
LCOA_18_60 = np.zeros((31, 301))

for i in range(43):
    LCOA_18_60[:, 0 + i * 7] = LCOA_CG_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 2 + i * 7] = LCOA_CG_CCS_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 1 + i * 7] = LCOA_SMR_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 3 + i * 7] = LCOA_SMR_CCS_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 4 + i * 7] = LCOA_PV_AWE_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 5 + i * 7] = LCOA_Wind_AWE_results_province_future_18_60_df.iloc[:, i]
    LCOA_18_60[:, 6 + i * 7] = LCOA_integrated_results_province_future_18_60_df.iloc[
        :, i
    ]

LCOA_18_60_df = pd.DataFrame(LCOA_18_60, index=Province)
